In [28]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import json
import requests
import re
from collections import OrderedDict

In [2]:
'''The website can be scraped only 100 startups at the time, each offset displaying 100 new startups.
So in order to scrape them, I had to go from 100 to 100 until the last offset, which is 2400, for a total of 2391 startups.
Increasing the limit int the URL didn't work and I couldn't find a workaround, this means that when new companies are added,
range must be increased.'''

r1 = requests.get('https://data.startups.be/api/actors/?category=Startup&actor_type=LegalEntity&limit=100').json()
all_r = []
all_r.append(r1)
for i in np.arange(100,2400,100):
    
    r = requests.get('https://data.startups.be/api/actors/?category=Startup&actor_type=LegalEntity&limit=100&offset='+str(i)).json()
    all_r.append(r)


In [141]:
'''Let's add all the interesting strings and values we are looking for, only the names and addresses are complete
for all startups so we will use them to fill as a baseline to complete are lists which will be the columns of our DataFrame'''

Nom_entreprise = []
Année_Création = []
CEO = []
Numéro_TVA = []
Mail = []
Téléphone = []
Site_web = []
Région = []
Adresse = []
count = []
for i in all_r:
    for x in i:
        for idx, value in enumerate(x):
            count.append(idx)
            if value =='name':
                Nom_entreprise.append(x[value])
                
                if 'vat_number' in x.keys():
                    Numéro_TVA.append(x['vat_number'])
                else: Numéro_TVA.append('None')
                    
                if 'url' in x.keys():
                    Site_web.append(x['url'])
                else: Site_web.append('None')
                    
            if value == 'address':
                Région.append(x[value]['admin1_name'])
                try:
                    Adresse.append(x[value]['street']+','+x[value]['number']+','+x[value]['zip']+' '+x[value]['locality'])
                except TypeError:
                    Adresse.append('Pas d\'adresse dans les données')
                    

In [142]:
### Sanity check, the columns need to be the same length to create a DataFrame ###

cols = [Site_web,Nom_entreprise,Numéro_TVA,Région,Adresse]
d = {'Site_web':Site_web,'Nom_entreprise':Nom_entreprise,'Numéro_TVA':Numéro_TVA,'Région':Région,'Adresse':Adresse}

for col in cols:
    print(len(col))

2391
2391
2391
2391
2391


In [143]:
Données = pd.DataFrame(d)

In [144]:
Données

,Site_web,Nom_entreprise,Numéro_TVA,Région,Adresse
0,http://bsit.com/parent,Bsit (Subleme),0632842351,Brussels Capital,"Louizalaan,120,1050 Brussels"
1,http://www.insidematters.eu,Inside Matters,0808162032,Flanders,"Ottergemsesteenweg-Zuid,808,9000 Gent"
2,http://enervalis.com,Enervalis,0509978587,Flanders,"Centrum-Zuid,1111,3530 Houthalen-Helchteren"
3,http://www.skarabee.com,Skarabee,0671844665,Flanders,"Everslaarstraat,63,9160 Lokeren"
4,https://lifepowr.co/,LifePower,0632577085,Flanders,"Museumstraat,39,2000 Antwerp"
5,https://amanote.com/fr/,Amaplex Software,0661870887,Wallonia,"Avenue de Hony,24,4130 Esneux"
6,http://aptus.be,Aptus,0568855609,Flanders,"Prins Boudewijnlaan,41,2650 Edegem"
7,https://dockflow.be,Dockflow,0693908801,Flanders,Pas d'adresse dans les données
8,http://tigerpower.eu,Tiger Power,0633892525,Flanders,"Molenweidestraat,24,9070 Destelbergen"
9,https://sunthetic.eu,Sunthetic,0745293461,Brussels Capital,"Aduatukersstraat,55,1040 Brussels"
